### Imports

In [19]:
import pandas as pd
import numpy as np
import sklearn as skit
import math

import geopy
from geopy.distance import VincentyDistance

from scipy import constants
constants.value



<function scipy.constants.codata.value(key)>

###    Funcao  PATH   

In [20]:

#calcula  distancia entre dois pontos B e M
def dist_BS_MS(latB, lonB, latM,lonM):
    coord_base = (latB, lonB)
    coord_mobi = (latM, lonM)
    return geopy.distance.vincenty(coord_base, coord_mobi).m


#calculo do Path loss para dada distancia
def freeSpacePL(dist, freq):
    return 32.44 + 20*np.log10(dist) + 20*np.log10(freq)
    
    # dei uma comentado pra usar da forma que está no git de robson, parecia estar certo, mas esse 147.55 eu não calculei
    # sei que é o resultado lá dos ganhos mas... depois debate isso
    
    #return 20*math.log(dist,10) + 20*math.log(freq,10) + 147.55221
    
    #conferir corretude em: https://www.everythingrf.com/rf-calculators/free-space-path-loss-calculator
#print(_FSPL(2, 1800000000))

def leeModelPL(dist, freq):
    n0 = 3.84
    p0 = 61.7
    txH = 50.0
    rxH = 1.5
    nf = 3 if freq > 850 else 2
    X2 = 2 if rxH > 3 else 1
    L1 = -20*np.log10(txH/30)
    L2 = -10*X2*np.log10(rxH/3)
    Lo = 50.3 + p0 - 10*n0*np.log10(1.61)-10*nf*np.log10(900)
    L = Lo + 10*n0*np.log10(dist)+10*nf*np.log10(freq)+L1+L2
    return L

def flatEarthPL(dist, freq):
    return 120 + (40*np.log10(dist)) - (20*np.log10(50.0) - (20*np.log10(1.5)))

                                  

In [21]:
#names=["pontoId","lat","lon","pathBTS1","pathBTS2","pathBTS3","pathBTS4","pathBTS5","pathBTS6","taBTS1","taBTS2","taBTS3","taBTS4","taBTS5","taBTS6"]
#BD referencia
# df_ref = pd.read_csv("CM/LocTreino_Equipe_6.csv")
df_ref = pd.read_csv("LocTreino_Equipe_6.csv")

df_ref = df_ref.drop(['pontoId','taBTS1','taBTS2','taBTS3','taBTS4','taBTS5','taBTS6'], axis = 1)

df_ref.head()



,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6
0,-8.076391,-34.895985,120.610000,104.380000,129.970000,123.620000,129.460000,132.080000
1,-8.072939,-34.895676,105.290000,121.523333,106.556667,108.756667,122.556667,126.123333
2,-8.076256,-34.899773,122.256667,110.423333,124.490000,135.256667,142.956667,142.556667
3,-8.064446,-34.889416,127.423333,132.656667,130.556667,131.856667,126.090000,137.723333
4,-8.068819,-34.888874,121.490000,136.423333,128.190000,122.723333,107.256667,120.323333


In [22]:
#Antenas
# df_antenas = pd.read_csv("CM/dados_BTSs.csv")
df_antenas = pd.read_csv("dados_BTSs.csv")
df_antenas


,name,lat,lon,band,bcch,eirp,bts
0,136_2,-8.068361,-34.892722,GSM1800,662,55.59,BTS-1
1,137_3,-8.075917,-34.894611,GSM1800,806,55.59,BTS-2
2,197_1,-8.076361,-34.908000,GSM1800,666,55.59,BTS-3
3,137_1,-8.075917,-34.894611,GSM1800,690,55.59,BTS-4
4,138_2,-8.066000,-34.889444,GSM1800,673,55.59,BTS-5
5,245_2,-8.064583,-34.894583,GSM1800,682,55.59,BTS-6


### lat's e lon's das extermidade do mapa
### largura e altura do grid

In [23]:
min_lat = df_ref['lat'].min()
max_lat = df_ref['lat'].max()
min_lon = df_ref['lon'].min()
max_lon = df_ref['lon'].max()

print("latitude maxima: {}".format(max_lat))
print("latitude minima: {}".format(min_lat))
print("***")
print("long maxima: {}".format(max_lon))
print("long minima: {}".format(min_lon))
print("***")
#coord -> (lat lon)
coord_min_min = (min_lat, min_lon)
coord_min_max = (min_lat, max_lon)
coord_max_max = (max_lat, max_lon)
coord_max_min = (max_lat, min_lon)

#rtg -> retangulo

grid_wd = math.ceil(geopy.distance.vincenty(coord_min_min, coord_min_max).m)
grid_ht = math.ceil(geopy.distance.vincenty(coord_min_min, coord_max_min).m)
print ("largura do grid em metros: {}".format(grid_wd))
print ("altura do grid em metros: {}".format(grid_ht))

latitude maxima: -8.059338
latitude minima: -8.077575999999999
***
long maxima: -34.885067
long minima: -34.90683
***
largura do grid em metros: 2399
altura do grid em metros: 2018


### altura, largura e quantidade de celulas
#cell_wd = prop x cell_ht
#cell_wd -> 100 metros

In [24]:
prop = grid_wd/grid_ht
#print(prop)
cell_ht = 50 #em metros
cell_wd = math.ceil(prop*cell_ht)

print("celula tem {} metros de largura".format(cell_wd +1))
print("celula tem {} metros de altura".format(cell_ht))

math.ceil
num_col = math.ceil(grid_wd/cell_wd)
num_row = math.ceil(grid_ht/cell_ht)
n_of_cells = int(num_col*num_row)

print("total de linhas: {}".format(num_row))
print("total de colunas: {}".format(num_col))

print("total de celulas: {}".format(n_of_cells))


celula tem 61 metros de largura
celula tem 50 metros de altura
total de linhas: 41
total de colunas: 40
total de celulas: 1640


### lat's e lon's de todos as celulas do grid

In [25]:

# calculo as coord do centro da primeira celula (mais abaixo e mais a esquerda):
origem = geopy.Point(min_lat, min_lon)
angulo = math.degrees(math.atan(cell_ht/cell_wd))
hipotenusa = math.hypot(cell_ht/2,cell_wd/2)
destination = VincentyDistance(meters=hipotenusa).destination(origem, angulo)
lat0, lon0 = destination.latitude, destination.longitude

#com base em lat0 e lon0 gerar as lats e as lons para as demais celulas:
latx,  lonx = lat0, lon0 
lats, lons = [], []
for l in range(num_row):
    for c in range(num_col):
        #add a lista
        lats.append(latx)
        lons.append(lonx)
        #proximas celulas a direita
        origem = geopy.Point(latx, lonx)
        destination = VincentyDistance(meters=cell_wd).destination(origem, math.degrees(0))
        latx, lonx = destination.latitude, destination.longitude
    #vai para a celula mais a esqueda da linha de cima
    origem = geopy.Point(lat0, lon0)
    destination = VincentyDistance(meters=(l+1)*cell_wd).destination(origem, math.degrees(math.pi/2))
    latx, lonx = destination.latitude, destination.longitude
print(len(lats))
    
data = np.array([lats, lons])
df_cells_coord = pd.DataFrame({'lat':data[0], 'lon':data[1]})
df_cells_coord.head()


1640


,lat,lon
0,-8.077305,-34.906603
1,-8.076762,-34.906603
2,-8.076220,-34.906603
3,-8.075677,-34.906603
4,-8.075135,-34.906603


### Calculo do path de todos as celulas

In [70]:
#def dist_BS_MS(latB, lonB, latM,lonM):
#def _FSPL(dist, freq):

#print (dist_BS_MS(min_lat, min_lon, min_lat, max_lon))
df_antenas.head()

# path_loss_BS1 = []
# path_loss_BS2 = []
# path_loss_BS3 = []
# path_loss_BS4 = []
# path_loss_BS5 = []
# path_loss_BS6 = []
path_loss_BS1 = {'freeSpacePL':[], 'leeModelPL':[], 'flatEarthPL': [], 'lat':[], 'lon':[]}
path_loss_BS2 = {'freeSpacePL':[], 'leeModelPL':[], 'flatEarthPL': [], 'lat':[], 'lon':[]}
path_loss_BS3 = {'freeSpacePL':[], 'leeModelPL':[], 'flatEarthPL': [], 'lat':[], 'lon':[]}
path_loss_BS4 = {'freeSpacePL':[], 'leeModelPL':[], 'flatEarthPL': [], 'lat':[], 'lon':[]}
path_loss_BS5 = {'freeSpacePL':[], 'leeModelPL':[], 'flatEarthPL': [], 'lat':[], 'lon':[]}
path_loss_BS6 = {'freeSpacePL':[], 'leeModelPL':[], 'flatEarthPL': [], 'lat':[], 'lon':[]}

print(len(df_cells_coord))
print(len(path_loss_BS1))
print(len(df_antenas))

n_cells = len(df_cells_coord)

#BS_1
for j in range(n_cells):
#     dist = dist_BS_MS(df_antenas.lat[0], df_antenas.lon[0], df_cells_coord.lat[j], df_cells_coord.lon[j])
#     pl1 = freeSpacePL(dist, 1800*(10**6))
#     path_loss_BS1.append(pl1)
    dist = dist_BS_MS(df_antenas.lat[0], df_antenas.lon[0], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl1 = freeSpacePL(dist, 1800*(10**6))
    path_loss_BS1['freeSpacePL'].append(pl1)
    pl2 = leeModelPL(dist, 1800*(10**6))
    path_loss_BS1['leeModelPL'].append(pl2)
    pl3 = flatEarthPL(dist, 1800*(10**6))
    path_loss_BS1['flatEarthPL'].append(pl3)
    
    lat1 = df_cells_coord['lat'][j]
    path_loss_BS1['lat'].append(lat1)
    lon1 = df_cells_coord['lon'][j]
    path_loss_BS1['lon'].append(lon1)
    
    
print(len(path_loss_BS1))

#BS_2
for j in range(n_cells):
#     dist = dist_BS_MS(df_antenas.lat[1], df_antenas.lon[1], df_cells_coord.lat[j], df_cells_coord.lon[j])
#     pl1 = freeSpacePL(dist, 1800*(10**6))
#     path_loss_BS2.append(pl1)
#     #pl2 = leeModelPL(dist, 1800*(10**6))
#     #path_loss_BS2.append(pl2)
#     #pl3 = flatEarthPL(dist, 1800*(10**6))
#     #path_loss_BS2.append(pl3)
    dist = dist_BS_MS(df_antenas.lat[1], df_antenas.lon[1], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl1 = freeSpacePL(dist, 1800*(10**6))
    path_loss_BS2['freeSpacePL'].append(pl1)
    pl2 = leeModelPL(dist, 1800*(10**6))
    path_loss_BS2['leeModelPL'].append(pl2)
    pl3 = flatEarthPL(dist, 1800*(10**6))
    path_loss_BS2['flatEarthPL'].append(pl3)
    
    lat2 = df_cells_coord['lat'][j]
    path_loss_BS2['lat'].append(lat2)
    lon2 = df_cells_coord['lon'][j]
    path_loss_BS2['lon'].append(lon2)
    
print(len(path_loss_BS2))

#BS_3
for j in range(n_cells):
#     dist = dist_BS_MS(df_antenas.lat[2], df_antenas.lon[2], df_cells_coord.lat[j], df_cells_coord.lon[j])
#     pl1 = freeSpacePL(dist, 1800*(10**6))
#     path_loss_BS3.append(pl1)
#     #pl2 = leeModelPL(dist, 1800*(10**6))
#     #path_loss_BS3.append(pl2)
#     #pl3 = flatEarthPL(dist, 1800*(10**6))
#     #path_loss_BS3.append(pl3)
    dist = dist_BS_MS(df_antenas.lat[2], df_antenas.lon[2], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl1 = freeSpacePL(dist, 1800*(10**6))
    path_loss_BS3['freeSpacePL'].append(pl1)
    pl2 = leeModelPL(dist, 1800*(10**6))
    path_loss_BS3['leeModelPL'].append(pl2)
    pl3 = flatEarthPL(dist, 1800*(10**6))
    path_loss_BS3['flatEarthPL'].append(pl3)
    
    lat3 = df_cells_coord['lat'][j]
    path_loss_BS3['lat'].append(lat3)
    lon3 = df_cells_coord['lon'][j]
    path_loss_BS3['lon'].append(lon3)
    
print(len(path_loss_BS3))

#BS_4
for j in range(n_cells):
#     dist = dist_BS_MS(df_antenas.lat[3], df_antenas.lon[3], df_cells_coord.lat[j], df_cells_coord.lon[j])
#     pl1 = freeSpacePL(dist, 1800*(10**6))
#     path_loss_BS4.append(pl1)
#     #pl2 = leeModelPL(dist, 1800*(10**6))
#     #path_loss_BS4.append(pl2)
#     #pl3 = flatEarthPL(dist, 1800*(10**6))
#     #path_loss_BS4.append(pl3)
    dist = dist_BS_MS(df_antenas.lat[3], df_antenas.lon[3], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl1 = freeSpacePL(dist, 1800*(10**6))
    path_loss_BS4['freeSpacePL'].append(pl1)
    pl2 = leeModelPL(dist, 1800*(10**6))
    path_loss_BS4['leeModelPL'].append(pl2)
    pl3 = flatEarthPL(dist, 1800*(10**6))
    path_loss_BS4['flatEarthPL'].append(pl3)
    
    lat4 = df_cells_coord['lat'][j]
    path_loss_BS4['lat'].append(lat4)
    lon4 = df_cells_coord['lon'][j]
    path_loss_BS4['lon'].append(lon4)

print(len(path_loss_BS4))

#BS_5
for j in range(n_cells):
#     dist = dist_BS_MS(df_antenas.lat[4], df_antenas.lon[4], df_cells_coord.lat[j], df_cells_coord.lon[j])
#     pl1 = freeSpacePL(dist, 1800*(10**6))
#     path_loss_BS5.append(pl1)
#     #pl2 = leeModelPL(dist, 1800*(10**6))
#     #path_loss_BS5.append(pl2)
#     #pl3 = flatEarthPL(dist, 1800*(10**6))
#     #path_loss_BS5.append(pl3)
    dist = dist_BS_MS(df_antenas.lat[4], df_antenas.lon[4], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl1 = freeSpacePL(dist, 1800*(10**6))
    path_loss_BS5['freeSpacePL'].append(pl1)
    pl2 = leeModelPL(dist, 1800*(10**6))
    path_loss_BS5['leeModelPL'].append(pl2)
    pl3 = flatEarthPL(dist, 1800*(10**6))
    path_loss_BS5['flatEarthPL'].append(pl3)
    
    lat5 = df_cells_coord['lat'][j]
    path_loss_BS5['lat'].append(lat5)
    lon5 = df_cells_coord['lon'][j]
    path_loss_BS5['lon'].append(lon5)
    
print(len(path_loss_BS5))

    
#BS_6
for j in range(n_cells):
#     dist = dist_BS_MS(df_antenas.lat[5], df_antenas.lon[5], df_cells_coord.lat[j], df_cells_coord.lon[j])
#     pl1 = freeSpacePL(dist, 1800*(10**6))
#     path_loss_BS6.append(pl1)
#     #pl2 = leeModelPL(dist, 1800*(10**6))
#     #path_loss_BS6.append(pl2)
#     #pl3 = flatEarthPL(dist, 1800*(10**6))
#     #path_loss_BS6.append(pl3)
    dist = dist_BS_MS(df_antenas.lat[5], df_antenas.lon[5], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl1 = freeSpacePL(dist, 1800*(10**6))
    path_loss_BS6['freeSpacePL'].append(pl1)
    pl2 = leeModelPL(dist, 1800*(10**6))
    path_loss_BS6['leeModelPL'].append(pl2)
    pl3 = flatEarthPL(dist, 1800*(10**6))
    path_loss_BS6['flatEarthPL'].append(pl3)
    
    lat6 = df_cells_coord['lat'][j]
    path_loss_BS6['lat'].append(lat6)
    lon6 = df_cells_coord['lon'][j]
    path_loss_BS6['lon'].append(lon6)

print(len(path_loss_BS6))

print('fim')


1640
5
6
5
5
5
5
5
5
fim


### Data frame com path dos centros das celulas

In [71]:
data_path = np.array([path_loss_BS1, path_loss_BS2, path_loss_BS3, path_loss_BS4, path_loss_BS5, path_loss_BS6])

data_path
df_cells_path = pd.DataFrame({'pathBTS1':data_path[0], 'pathBTS2':data_path[1], 'pathBTS3':data_path[2], 'pathBTS4':data_path[3], 'pathBTS5':data_path[4], 'pathBTS6':data_path[5]})
df_cells_path.head()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6
flatEarthPL,"[219.96338725328374, 219.6568705176404, 219.35...","[214.50540476016653, 214.43240442149485, 214.3...","[180.322755317784, 177.73156261784277, 177.127...","[214.505403902564, 214.43240355665324, 214.394...","[223.76200016683458, 223.5109757500332, 223.26...","[220.9877235484345, 220.59469149856622, 220.20..."
freeSpacePL,"[282.75593118151136, 282.6026728136897, 282.45...","[280.0269399349528, 279.99043976561694, 279.97...","[262.9356152137615, 261.6400188637909, 261.338...","[280.0269395061515, 279.99043933319615, 279.97...","[284.6552376382868, 284.52972542988607, 284.40...","[283.26809932908674, 283.0715833041526, 282.87..."
lat,"[-8.077304742881273, -8.076762228758346, -8.07...","[-8.077304742881273, -8.076762228758346, -8.07...","[-8.077304742881273, -8.076762228758346, -8.07...","[-8.077304742881273, -8.076762228758346, -8.07...","[-8.077304742881273, -8.076762228758346, -8.07...","[-8.077304742881273, -8.076762228758346, -8.07..."
leeModelPL,"[416.866234867144, 416.5719788009264, 416.2863...","[411.62657167375147, 411.55649134862665, 411.5...","[378.8112282090642, 376.3236832171207, 375.744...","[411.62657085045305, 411.55649051837867, 411.5...","[420.5129032641528, 420.2719198240234, 420.036...","[417.8495977104887, 417.4722869426152, 417.094..."
lon,"[-34.90660318588656, -34.90660318588656, -34.9...","[-34.90660318588656, -34.90660318588656, -34.9...","[-34.90660318588656, -34.90660318588656, -34.9...","[-34.90660318588656, -34.90660318588656, -34.9...","[-34.90660318588656, -34.90660318588656, -34.9...","[-34.90660318588656, -34.90660318588656, -34.9..."


### Dataframe com coord e path dos centros das celulas

In [64]:
df_cell_coords_paths = df_cells_coord.join(df_cells_path, lsuffix='_df_cells_coord', rsuffix='_df_cells_path')

print(len(df_cell_coords_paths))

df_cell_coords_paths.head()

1640


,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6
0,-8.077305,-34.906603,NaN,NaN,NaN,NaN,NaN,NaN
1,-8.076762,-34.906603,NaN,NaN,NaN,NaN,NaN,NaN
2,-8.076220,-34.906603,NaN,NaN,NaN,NaN,NaN,NaN
3,-8.075677,-34.906603,NaN,NaN,NaN,NaN,NaN,NaN
4,-8.075135,-34.906603,NaN,NaN,NaN,NaN,NaN,NaN


### K-fold cross validation

In [29]:
features = ['pathBTS1', 'pathBTS2', 'pathBTS3', 'pathBTS4', 'pathBTS5', 'pathBTS6']
to_predict=['lat', 'lon']

### ERRO

In [42]:
def calculate_errors(pathloss_measures_df, pathloss_model_df):
    squaredError = ((pathloss_measures_df - pathloss_model_df) ** 2)
    rmse = squaredError.mean(axis=0) ** (0.5)

    return rmse

In [43]:
import numpy as np
from geographiclib.geodesic import Geodesic,Math

def distanceGeoLibArr(lat1,lon1,lat2,lon2,extract):
        if (np.isscalar(lat1) and np.isscalar(lat2)):
            return distanceGeoLib(lat1,lon1,lat2,lon2,extract)
        elif (np.isscalar(lat1) and (not np.isscalar(lat2))):
            resp=[]
            for i in range(len(lat2)):
                r = distanceGeoLib(lat1,lon1,lat2[i],lon2[i],extract)
                resp.append(r)
            return resp
        elif (not np.isscalar(lat1) and (not np.isscalar(lat2))):
            resp=[]
            for i in range(len(lat1)):
                r = distanceGeoLib(lat1[i],lon1[i],lat2[i],lon2[i],extract)
                resp.append(r)
            return resp

def distanceGeoLib(lat1,lon1,lat2,lon2,extract):
    MASK=Geodesic.DISTANCE|Geodesic.AZIMUTH|Geodesic.REDUCEDLENGTH
    gab=Geodesic.WGS84.Inverse(lat1,lon1,lat2,lon2,MASK)
    resp = extract(gab)
    return resp

def distanceInKm(lat1,lon1,lat2,lon2):
    return distanceGeoLibArr(lat1,lon1,lat2,lon2,lambda g:g['s12']/1000)

def AzimuthAtoB(lata,lona,latb,lonb):
    return distanceGeoLibArr(lata,lona,latb,lonb,lambda g:g['azi1'])
    

In [44]:
def distances_dataframe(erbs, measures):
    users_lat = measures['lat'].values.reshape((1,measures.shape[0]))[0]
    users_lon = measures['lon'].values.reshape((1,measures.shape[0]))[0]
    indexes = erbs['bts'].values.reshape((1,6))[0]    

    distances_dict = {}

    for i in range(erbs.shape[0]):
        distances_dict[indexes[i]] = calculate_distances(erbs.iloc[i], users_lat, users_lon)

    return pd.DataFrame(data = distances_dict)


def calculate_distances(erb, users_lat, users_lon):
    return distanceInKm(erb['lat'], erb['lon'], users_lat, users_lon)


In [45]:
def calculate_pathloss_measures(erbs, measures):
    erbs_pt = erbs['eirp'].values.reshape((1,6))
    indexes = erbs['bts'].values.reshape((1,6))[0]
    
    pathloss_dict = {}
    
    for i in range(measures.shape[0]):
        user_pr = measures.iloc[i][2:].values.reshape((1,6))
        pathloss = erbs_pt - user_pr
        pathloss_dict[i] = pathloss[0]
    
    return pd.DataFrame(data = pathloss_dict, index = indexes).T

In [46]:
dis = distances_dataframe(df_antenas, df_ref)
pathLossMedicao = calculate_pathloss_measures(df_antenas, df_ref)

In [47]:
pathLoss = df_cell_coords_paths.drop(['lat','lon'],axis = 1)

pathLoss.rename(columns={'pathBTS1': 'BTS-1', 'pathBTS2' :'BTS-2', 'pathBTS3': 'BTS-3', 'pathBTS4': 'BTS-4', 'pathBTS5': 'BTS-5',
                         'pathBTS6': 'BTS-6'}, inplace=True)

erro = calculate_errors(pathLossMedicao,pathLoss)

In [48]:
pathLossMedicao.head()

,BTS-1,BTS-2,BTS-3,BTS-4,BTS-5,BTS-6
0,-65.020000,-48.790000,-74.380000,-68.030000,-73.870000,-76.490000
1,-49.700000,-65.933333,-50.966667,-53.166667,-66.966667,-70.533333
2,-66.666667,-54.833333,-68.900000,-79.666667,-87.366667,-86.966667
3,-71.833333,-77.066667,-74.966667,-76.266667,-70.500000,-82.133333
4,-65.900000,-80.833333,-72.600000,-67.133333,-51.666667,-64.733333


In [49]:
pathLoss.head()

,BTS-1,BTS-2,BTS-3,BTS-4,BTS-5,BTS-6
0,282.755931,280.026940,262.935615,280.026940,284.655238,283.268099
1,282.602673,279.990440,261.640019,279.990439,284.529725,283.071583
2,282.453918,279.971553,261.338217,279.971552,284.406924,282.874927
3,282.310338,279.970510,262.232614,279.970510,284.287175,282.678543
4,282.172636,279.987325,263.788433,279.987325,284.170836,282.482899


In [50]:
erro


BTS-1    337.352455
BTS-2    342.142919
BTS-3    349.585596
BTS-4    338.902738
BTS-5    343.699874
BTS-6    345.386138
dtype: float64

In [51]:
dis.head()

,BTS-1,BTS-2,BTS-3,BTS-4,BTS-5,BTS-6
0,0.958131,0.160263,1.324330,0.160263,1.356616,1.314990
1,0.601945,0.349611,1.410129,0.349611,1.029920,0.931920
2,1.168916,0.570194,0.906877,0.570194,1.607065,1.412010
3,0.565946,1.391861,2.435669,1.391861,0.171895,0.569776
4,0.427182,1.008004,2.267161,1.008004,0.318048,0.784524


In [52]:
from sklearn.model_selection import train_test_split

X = df_cell_coords_paths.drop(['lat','lon'], axis = 1)
y = df_cell_coords_paths.iloc[:, 0:2]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [53]:
X_train.head()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6
111,281.933435,283.936793,282.538640,283.936793,282.997808,279.738808
1200,277.762365,271.563528,283.376137,271.563529,279.508304,280.979820
1300,270.481183,279.168328,284.950615,279.168328,252.792104,273.533599
508,278.089418,281.846755,282.659590,281.846755,279.467392,273.984344
302,278.821758,280.917948,280.115563,280.917948,280.909339,276.717218


In [54]:
y_train.head()

,lat,lon
111,-8.060487,-34.905514
1200,-8.077304,-34.890273
1300,-8.066454,-34.889184
508,-8.062114,-34.900071
302,-8.065369,-34.902793


In [55]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=3)

In [56]:
knn.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=3, p=2,
          weights='uniform')

In [76]:
# X_train.head()

,lat,lon
1459,-8.066997,-34.887006
973,-8.070252,-34.893539
1415,-8.069167,-34.887551
65,-8.063742,-34.906059
1478,-8.056689,-34.887006


In [57]:
from sklearn.metrics import mean_squared_error
y_pred = knn.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
print('rmse:{}'.format(rmse))

rmse:0.0002742810393798081


In [78]:
y_test.head()

,lat,lon
1459,-8.066997,-34.887006
973,-8.070252,-34.893539
1415,-8.069167,-34.887551
65,-8.063742,-34.906059
1478,-8.056689,-34.887006


In [79]:
y_pred

array([[ -8.06717735, -34.8871879 ],
       [ -8.06989018, -34.8939016 ],
       [ -8.06898574, -34.88736935],
       [ -8.06374189, -34.90624028],
       [ -8.05668873, -34.8871879 ],
       [ -8.07459201, -34.8949903 ],
       [ -8.06410311, -34.88736935],
       [ -8.07694273, -34.88990967],
       [ -8.06808198, -34.90297416],
       [ -8.05686967, -34.88936531],
       [ -8.07169831, -34.88736935],
       [ -8.05994403, -34.89226853],
       [ -8.07260284, -34.89698627],
       [ -8.07404965, -34.90406287],
       [ -8.06229478, -34.88845806],
       [ -8.069167  , -34.9013411 ],
       [ -8.07477282, -34.89408305],
       [ -8.06934766, -34.89353869],
       [ -8.07404963, -34.90188545],
       [ -8.07423044, -34.89970804],
       [ -8.07079456, -34.90406287],
       [ -8.07278377, -34.90242981],
       [ -8.06645444, -34.90242981],
       [ -8.07296412, -34.88628064],
       [ -8.07477282, -34.89408305],
       [ -8.07658115, -34.89263144],
       [ -8.07043273, -34.8949903 ],
 

In [112]:
import math

def calc_dist(lat1, lon1, lat2, lon2):
    R = 6378.137; # Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000 # meters

#calcula  distancia entre dois pontos B e M

def dist_BS_MS(latB, lonB, latM,lonM):
    coord_base = (latB, lonB)
    coord_mobi = (latM, lonM)
    return geopy.distance.vincenty(coord_base, coord_mobi).m

u = 0
dists = {'dists':[]}

while u<len(y_pred):
    lat1 = y_pred[u][0]
    lon1 = y_pred[u][1]
    lat2 = y_test['lat'].iloc[u]
    lon2 = y_test['lon'].iloc[u]
    
#     dists['dists'].append(calc_dist(lat1, lon1, lat2, lon2))
    dists['dists'].append(dist_BS_MS(lat1,lon1,lat2,lon2))
    u = u+1
    
df_dists = pd.DataFrame(dists).head()
df_dists

,dists
0,28.285296
1,56.568195
2,28.285228
3,20.000666
4,20.001013


In [106]:
# print(dists)

In [111]:
rms = np.sqrt(np.mean(list(map(lambda x: x**2, dists['dists']))))
print(rms)

42.82733617661828
